## 한글 문서의 분류
다음무비(http://movie.daum.net)로부터 crawl한 영화리뷰를 이용하여 분류 연습<br>
영화리뷰와 영화의 제목을 학습해서 주어진 리뷰내용으로 어떤 영화에 대한 리뷰인지를 예측하고자 함
인스티즈포털(https://www.instiz.net/name_enter?category=0)에서 크롤링한 아이돌 검색 별 게시글 제목을 이용하여 분류연습<br>
게시글 제목과 아이돌 분류를 학습해서 어떤 아이돌에 대한 게시물 제목인지 예측
### data file 내용
방탄소년단, 엑소, 트와이스, 아이즈원에 대한 게시글제목 데이터임
총 2280개의 데이터
<U+00...>등의 필요없는 문자열을 제거하려고 했으나, R과 파이썬을 전부 이용해도 지워지지 않고, 엑셀에서 수동적으로 수정해서 저장하면 계속 인코딩 문제가 발생했음.

In [69]:
import csv

text = []
y = []
with open('아이돌.csv', encoding='utf-8') as csvfile:
    csvreader = csv.reader(csvfile)
    for row in csvreader:
        #print(row)
        if row: #그 줄에 내용이 있는 경우에만
            text.append(row[0]) #게시글 제목을 text 리스트에 추가
            y.append(row[1]) #아이돌 그룹명을 text 리스트에 추가

In [70]:
print('Num of samples: {}'.format(len(text)))
print('아이돌 게시글: {}'.format(set(y)))

Num of samples: 2280
아이돌 게시글: {'BTS', 'exo', 'twice', 'izone'}


In [71]:
from sklearn.model_selection import train_test_split

# split data and labels into a training and a test set
X_train, X_test, y_train, y_test = train_test_split(text, y, random_state=0)
# 비율을 지정하지 않으면 75:25로 분할됨

In [89]:
len(X_train)

1710

In [73]:
from konlpy.tag import Okt #konlpy에서 Twitter 형태소 분석기를 import
#from konlpy.tag import Twitter #konlpy에서 Twitter 형태소 분석기를 import
twitter_tag = Okt()
#twitter_tag = Twitter()

In [74]:
print(twitter_tag.morphs(X_train[1])) #둘째 게시글제목에 대해 형태소 단위로 tokenize

['<', 'U', '+', '0001', 'F', '496', '>', '월일', '에', '뜬', '쯔위', '컨셉', '사진', '움짤', '모음', '<', 'U', '+', '0001', 'F', '496', '>']


In [75]:
twitter_tag.nouns(X_train[1]) #둘째 게시글 제목에서 명사만 추출

['월일', '쯔위', '컨셉', '사진', '움짤', '모음']

In [76]:
def twit_tokenizer(text): # Twitter 형태소 분석기의 명사추출함수를 tokenizer 함수로 사용
    return twitter_tag.nouns(text)

In [78]:
X_test[:10] #test data에서 앞 10개를 출력

['혹쉬 최근에 입덕한 원스들 있니 ',
 '멜프라니ㅣ이ㅣㅠㅠㅠㅠㅠ',
 '아쪼 인스타 봤니   ',
 '나 개 다 보냈는데 지금 혹시 몰라 다시 누르니까 또 보내져',
 '유리 ost 오늘 공개래ㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠ',
 '<U+0001F525><U+0001F525><U+0001F525><U+0001F525><U+0001F525>ㅊㅇㄷ 다들 하고 있지  탄소들<U+0001F618><U+0001F525><U+0001F525><U+0001F525><U+0001F525>',
 '눈들 얼마나 있어 ',
 '아아앜 ㅠㅠㅠㅠ 왜 매번 다른 사람들이 빠지냐구ㅠㅠㅠㅠ',
 '이번엔 티저 때문에 뮤비가 인기급상승동영상에 안 올라가는 일 없길 <U+0001F64F>',
 '공지<U+0001F4E2><U+0001F4E2><U+0001F4E2><U+0001F4E2>']

In [79]:
clf.predict(X_test_tfidf[:10]) # test data의 예측내용

array(['twice', 'twice', 'exo', 'BTS', 'izone', 'BTS', 'izone', 'BTS',
       'twice', 'BTS'], dtype='<U5')

In [80]:
print(y_test[:10]) # test data 실제 아이돌 분류

['twice', 'twice', 'izone', 'BTS', 'izone', 'BTS', 'izone', 'BTS', 'twice', 'exo']


### 성능을 개선하기 위한 노력

In [81]:
# morphs()는 명사 외에도 모든 형태소를 포함
print(twitter_tag.morphs(X_train[1]))

['<', 'U', '+', '0001', 'F', '496', '>', '월일', '에', '뜬', '쯔위', '컨셉', '사진', '움짤', '모음', '<', 'U', '+', '0001', 'F', '496', '>']


In [82]:
tfidf = TfidfVectorizer(tokenizer=twitter_tag.morphs, min_df=2) # 명사 대신 모든 형태소를 사용
#tfidf = TfidfVectorizer(tokenizer=twit_tokenizer, min_df=3, max_df=0.90, max_features=1000, use_idf=True, sublinear_tf=True)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

clf = LogisticRegression()
clf.fit(X_train_tfidf, y_train)
print('Train score', clf.score(X_train_tfidf, y_train))
print('Test score', clf.score(X_test_tfidf, y_test))
print(X_train_tfidf.shape)
#명사만 사용한 것에 비해 train score는 상승, test score는 하락

Train score 0.8830409356725146
Test score 0.6491228070175439
(1710, 1243)


C:\Users\YeonJI\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\YeonJI\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [83]:
print(twitter_tag.pos(X_train[1], norm=True, stem=True)) #pos()는 형태소와 품사를 함께 제공

[('<', 'Punctuation'), ('U', 'Alpha'), ('+', 'Punctuation'), ('0001', 'Number'), ('F', 'Alpha'), ('496', 'Number'), ('>', 'Punctuation'), ('월일', 'Noun'), ('에', 'Josa'), ('뜨다', 'Verb'), ('쯔위', 'Noun'), ('컨셉', 'Noun'), ('사진', 'Noun'), ('움짤', 'Noun'), ('모음', 'Noun'), ('<', 'Punctuation'), ('U', 'Alpha'), ('+', 'Punctuation'), ('0001', 'Number'), ('F', 'Alpha'), ('496', 'Number'), ('>', 'Punctuation')]


In [84]:
def twit_tokenizer2(text): #전체를 다 사용하는 대신, 명사, 동사, 형용사를 사용
    target_tags = ['Noun', 'Verb', 'Adjective']
    result = []
    for word, tag in twitter_tag.pos(text, norm=True, stem=True):
        if tag in target_tags:
            result.append(word)
#            result.append('/'.join([word, tag]))
    return result

In [86]:
print(twit_tokenizer2(X_train[1])) # 사용 예

['월일', '뜨다', '쯔위', '컨셉', '사진', '움짤', '모음']


In [87]:
tfidf = TfidfVectorizer(tokenizer=twit_tokenizer2, min_df=2) #명사, 동사, 형용사를 이용하여 tfidf 생성
#tfidf = TfidfVectorizer(tokenizer=twit_tokenizer, min_df=3, max_df=0.90, max_features=1000, use_idf=True, sublinear_tf=True)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

clf = LogisticRegression()
clf.fit(X_train_tfidf, y_train)
print('Train score', clf.score(X_train_tfidf, y_train))
print('Test score', clf.score(X_test_tfidf, y_test))
print(X_train_tfidf.shape)
# 현재까지 중에서 test score가 가장 뛰어남

Train score 0.8619883040935673
Test score 0.6631578947368421
(1710, 911)


In [90]:
# 모든 형태소를 다 사용하고 품사를 알 수 있도록 하면?
def twit_tokenizer3(text):
    #target_tags = ['Noun', 'Verb', 'Adjective']
    result = []
    for word, tag in twitter_tag.pos(text, norm=True, stem=True):
        result.append('/'.join([word, tag])) #단어의 품사를 구분할 수 있도록 함
    return result

In [92]:
tfidf = TfidfVectorizer(tokenizer=twit_tokenizer3, min_df=2)
#tfidf = TfidfVectorizer(tokenizer=twit_tokenizer, min_df=3, max_df=0.90, max_features=1000, use_idf=True, sublinear_tf=True)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

clf = LogisticRegression()
clf.fit(X_train_tfidf, y_train)
print('Train score', clf.score(X_train_tfidf, y_train))
print('Test score', clf.score(X_test_tfidf, y_test))
print(X_train_tfidf.shape)
#Train score 상승, #Test score 하락

Train score 0.87953216374269
Test score 0.6578947368421053
(1710, 1197)


In [21]:
# train score가 높으므로 ridge를 쓰면 어떨까?
from sklearn.linear_model import RidgeClassifier
ridge_clf = RidgeClassifier(alpha = 1)
ridge_clf.fit(X_train_tfidf, y_train)
print('Train set score: {:.3f}'.format(ridge_clf.score(X_train_tfidf, y_train)))
print('Test set score: {:.3f}'.format(ridge_clf.score(X_test_tfidf, y_test)))
# train score가 올라가는 현상이 벌어짐
# test score도 함께 증가

Train set score: 0.944
Test set score: 0.676


In [93]:
#lasso를 쓰면?
from sklearn.linear_model import LogisticRegression
import numpy as np
lasso_clf = LogisticRegression(penalty='l1', solver='liblinear')
lasso_clf.fit(X_train_tfidf, y_train)
print('Train set score: {:.3f}'.format(lasso_clf.score(X_train_tfidf, y_train)))
print('Test set score: {:.3f}'.format(lasso_clf.score(X_test_tfidf, y_test)))
print('Used features count: {}'.format(np.sum(lasso_clf.coef_ != 0)), 'out of', X_train_tfidf.shape[1])

Train set score: 0.773
Test set score: 0.649
Used features count: 387 out of 1197


In [94]:
#lsa를 쓰면?
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=239, n_iter=7, random_state=42) #압축할 component의 수 지정
svd.fit(X_train_tfidf)  
print(svd.explained_variance_ratio_)  #계산된 각 component가 설명하는 분산의 비율
print(svd.explained_variance_ratio_.sum())  #선택된 component들이 설명하는 분산의 합 -> 선택한 component의 수에 따라 달라짐
print(svd.singular_values_) 
print(svd.components_.shape)

[0.0530101  0.00714228 0.00929344 0.00910356 0.00847454 0.00790744
 0.0074683  0.00726786 0.00715722 0.00707493 0.0068332  0.00650216
 0.00628227 0.00615433 0.0060278  0.00590501 0.00577303 0.00553344
 0.00543666 0.00540344 0.00521521 0.00515999 0.00502858 0.00489193
 0.00487598 0.00483086 0.00468443 0.0046638  0.0044988  0.00439197
 0.00426873 0.00427881 0.00415167 0.00402148 0.00399458 0.00392669
 0.0038507  0.00381235 0.00377764 0.0037254  0.00363985 0.00362168
 0.00347248 0.00347083 0.00340898 0.00334986 0.00331703 0.00326031
 0.00320145 0.00320096 0.00311225 0.00311543 0.00309848 0.0030782
 0.00304043 0.00301814 0.00300273 0.002998   0.00298492 0.00293321
 0.00288272 0.00284673 0.00281811 0.00280483 0.00277779 0.00274119
 0.00273441 0.00272243 0.00271764 0.00268047 0.00266405 0.00263714
 0.00261629 0.00260627 0.00258516 0.00256405 0.00255967 0.00253091
 0.00252215 0.00249661 0.00245966 0.00245895 0.00243202 0.00242694
 0.00240047 0.00238942 0.00237139 0.00236128 0.00233396 0.00231

In [95]:
X_train_svd = svd.transform(X_train_tfidf) #선택된 component를 이용하여 2,000개의 feature로부터 feature extract (dimension reduce)
X_test_svd = svd.transform(X_test_tfidf)

from sklearn.linear_model import LogisticRegression
SVD_clf = LogisticRegression()
SVD_clf.fit(X_train_svd, y_train)
print('Train set score: {:.3f}'.format(SVD_clf.score(X_train_svd, y_train)))
print('Test set score: {:.3f}'.format(SVD_clf.score(X_test_svd, y_test)))

C:\Users\YeonJI\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Train set score: 0.774
Test set score: 0.621


In [96]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(tokenizer=twit_tokenizer2, min_df=2).fit(X_train) #tfidf와 동일하게 max_feature를 제한하여 학습
X_train_cv = cv.transform(X_train) # train set을 변환
print('Train set dimension:', X_train_cv.shape) # 36310 대신 2000이 된 것을 확인
X_test_cv = cv.transform(X_test) # test set을 변환
print('Test set dimension:', X_test_cv.shape)

from sklearn.naive_bayes import MultinomialNB
NB_clf = MultinomialNB()
NB_clf.fit(X_train_cv, y_train)
print('Train set score: {:.3f}'.format(NB_clf.score(X_train_cv, y_train)))
print('Test set score: {:.3f}'.format(NB_clf.score(X_test_cv, y_test)))

Train set dimension: (1710, 911)
Test set dimension: (570, 911)
Train set score: 0.833
Test set score: 0.654
